# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
import pandas as pd

recipes= pd.read_csv('recipes_sample.csv', sep = ',')
reviews = pd.read_csv('reviews_sample.csv', sep = ',')

recipesdf = pd.DataFrame(recipes, columns = ['id','name','minutes','submitted','description','n_ingredients'])
reviewsdf = pd.DataFrame(reviews)




2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [3]:
from openpyxl import Workbook
from openpyxl import load_workbook

recipesdf5= recipesdf.head(round(30000*0.05))
reviewsdf5= reviewsdf.head(round(126696*0.05))

sheets = {'Рецепты': recipesdf5, 'Отзывы': reviewsdf5}
writer = pd.ExcelWriter('./recipes.xlsx', engine='xlsxwriter')

for sheet_name in sheets.keys():
    sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

writer.save()

C:\Users\gdoq1\AppData\Local\Temp\ipykernel_7280\3985232462.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [4]:
import xlwings as xw

wb = xw.Book('recipes.xlsx')

sht = wb.sheets['Рецепты']

seconds = recipesdf['minutes'].tolist()
for i in range(len(seconds)):
    seconds[i] = seconds[i]*60
    
    
    
sht.range('G1').value = 'seconds_assign'
sht.range('G2:G1501').options(transpose = True).value = seconds

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [5]:
sht.range('H1').value = 'seconds_formula'
sht.range('H2:H1501').options(transpose = True).value = '=C2*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [6]:
sht.range('G1').font.bold = True
sht.range('G1:G1501').api.HorizontalAlignment = -4108
sht.range('H1').font.bold = True
sht.range('H1:H1501').api.HorizontalAlignment = -4108

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [7]:
print(sht.range(1,1).value)

for i in range(2,1501):
    if sht.range(i,3).value < 5:
        sht.range(i,3).color = (0,255,0)
    elif sht.range(i,3).value >= 5 and sht.range(i,3).value < 10:
        sht.range(i,3).color = (255,255,0)
    elif sht.range(i,3).value >= 10:
        sht.range(i,3).color = (255,0,0)



id


7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [21]:
sht.range('I1').value = 'n_reviews'
sht.range('I1').font.bold = True
sht.range('I1:I1501').api.HorizontalAlignment = -4108

dict = {}
ids = []

for i in range(2,1501):
    ids.append(int(sht.range(i,1).value))

dict2 = dict.fromkeys(ids)
ids2 = reviewsdf['recipe_id'].tolist()


In [35]:
from collections import Counter

a = Counter(ids2)

dict2

for i in range(2,1501):
    sht.range(i,9).value = dict2[sht.range(i,1).value]


## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 